In [17]:
#Specify the CBSA of interest
cbsa_code = "17140"

In [18]:
import requests
import pandas as pd
import numpy as np
#!pip install ipfn
from ipfn import ipfn
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from pandas.core.ops.array_ops import na_logical_op
import warnings
from google.colab import drive
import os
from google.colab import drive

drive.mount('/content/drive')
# Mount Google Drive to access your files

# Please specify your directory root
base_dir = '/content/drive/MyDrive/Yangyang_Tayo/'

# Specify the geographical area of interest. Use NBER to identify the correct state code and county code(s)
cross_table = pd.read_csv(base_dir + "Data/cbsa_ct_cross.csv")
# Read a CSV file that maps CBSA codes to state and county information

table = cross_table[cross_table["cbsacode"] == int(cbsa_code)]
# Filter the table to find the entry for the specified CBSA code

area = table["cbsatitle"].values[0]
# Get the name of the geographical area from the table
print(area)

state_code = table["fipsstatecode"].values
formatted_strings = [f"{value:02d}" for value in state_code]
state_codes_str = ",".join(map(str, formatted_strings))
# Extract state codes and format them as two-digit strings, then join them as a comma-separated string

county_code = table["fipscountycode"].values
formatted_strings = [f"{value:03d}" for value in county_code]
county_codes_str = ",".join(map(str, formatted_strings))
# Extract county codes and format them as three-digit strings, then join them as a comma-separated string

region_df = table[["fipsstatecode", "fipscountycode"]]
# Create a DataFrame with state and county codes

def format_region(row):
    return f"state:{row['fipsstatecode']:02d} county:{row['fipscountycode']:03d}"
# Define a function to format state and county information

region_df['region'] = region_df.apply(format_region, axis=1)
# Apply the format_region function to create a 'region' column in the DataFrame

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cincinnati, OH-KY-IN


<ipython-input-18-af1de06fa294>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_df['region'] = region_df.apply(format_region, axis=1)


# 2019

## Synthetic Population Generation

In [19]:
#API calling function for ACS/PUMS data
def extract_census_data(url, group, region_df):
    # Define the API request parameters
    url = url
    params = {
        "get": f"group({group})",  # Specify the data group to retrieve
        "for": "tract:*",  # Specify the geographic level (tract in this case)
        "key": "9fe29c82d32372c3d1629631488612744bd7fe2f"  # Authentication key
    }

    # Initialize an empty DataFrame to store the data
    final_df = pd.DataFrame()

    # Iterate through the regions and make API calls
    for region in tqdm(region_df['region']):
        params["in"] = region  # Set the region parameter in the API request
        response = requests.get(url, params=params)  # Send the API request

        if response.status_code == 200:  # Check if the request was successful
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            final_df = pd.concat([final_df, df], ignore_index=True)
            # Concatenate the retrieved data to the final DataFrame
            # This loop fetches data for each region and adds it to the final DataFrame
            # Print a success message for the region
        else:
            print(f"Failed to retrieve data for {region}. Status code: {response.status_code}")
            # Print an error message if the request was unsuccessful

    return final_df  # Return the combined DataFrame with all the retrieved data

In [20]:
# Define the API URL and parameters. Use those to get the marginal sums for different social factors.

# Extract data related to social factors for group "S0101"
df = extract_census_data("https://api.census.gov/data/2019/acs/acs5/subject", "S0101", region_df)
data = df.set_index("NAME")

# Calculate the senior "T" and non-senior population "F"
data["T"] = (data[["S0101_C01_015E"]].values).astype(int) + (data[["S0101_C01_016E"]].values).astype(int) + (data[["S0101_C01_017E"]].values).astype(int) + (data[["S0101_C01_018E"]].values).astype(int) + (data[["S0101_C01_019E"]].values).astype(int)
data["F"] = (data[["S0101_C01_001E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Transpose data to create a DataFrame for age data
age_data = data[["T", "F"]].T

# Calculate the female "T" and non-female population "F"
data["T"]=(data[["S0101_C05_001E"]].values).astype(int)
data["F"]=(data[["S0101_C01_001E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Transpose data to create a DataFrame for sex data
sex_data = data[["T","F"]].T

# Extract data related to social factors for group "S1101"
df = extract_census_data("https://api.census.gov/data/2019/acs/acs5/subject", "S1101", region_df)
data = df.set_index("NAME")

# Calculate the total childed-household "T" and non-childed-household population "F"
data["T"] = (data[["S1101_C01_005E"]].values).astype(int)
data["F"] = (data[["S1101_C01_001E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Transpose data to create a DataFrame for citizenship status data
cyc_data = data[["T", "F"]].T

# Extract data related to social factors for group "S1501"
df = extract_census_data("https://api.census.gov/data/2019/acs/acs5/subject", "S1501", region_df)
data = df.set_index("NAME")

# Calculate the college "T" and non-college "F"
data["T"] = (data[["S1501_C01_005E"]].values).astype(int) + (data[["S1501_C01_015E"]].values).astype(int)
data["F"] = (data[["S1501_C01_001E"]].values).astype(int) + (data[["S1501_C01_006E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Transpose data to create a DataFrame for education data
edu_data = data[["T", "F"]].T

# Extract data related to social factors for group "S1901"
df = extract_census_data("https://api.census.gov/data/2019/acs/acs5/subject", "S1901", region_df)
data = df.set_index("NAME")

# Calculate high income "H," middle income "M," and low income "L"
data["H"] = (data[["S1901_C01_001E"]].values).astype(int) * ((data[["S1901_C01_010E"]].values).astype(float) + (data[["S1901_C01_011E"]].values).astype(float)) / 100
data["M"] = (data[["S1901_C01_001E"]].values).astype(int) * ((data[["S1901_C01_008E"]].values).astype(float) + (data[["S1901_C01_009E"]].values).astype(float)) / 100
data["L"] = (data[["S1901_C01_001E"]].values).astype(int) - (data[["H"]].values).astype(float)

# Transpose data to create a DataFrame for income data
inc_data = data[["L", "M", "H"]].T

# Extract data related to social factors for group "B08014"
df = extract_census_data("https://api.census.gov/data/2019/acs/acs5", "B08014", region_df)
data = df.set_index("NAME")

# Calculate the total population "T" and population with access to a car "F"
data["T"] = (data[["B08014_001E"]].values).astype(int) - (data[["B08014_002E"]].values).astype(int)
data["F"] = (data[["B08014_002E"]].values).astype(int)

# Transpose data to create a DataFrame for car ownership data
car_data = data[["T", "F"]].T

100%|██████████| 15/15 [00:10<00:00,  1.37it/s]


In [21]:
# Define the API endpoint and parameters to retrieve Census PUMS (Public Use Microdata Sample) data
api_endpoint = "https://api.census.gov/data/2019/acs/acs1/pums"
params = {
    "get": "SEX,SCHL,HHT,VEH,HINCP,AGEP",
    "for": "state:" + state_codes_str,  # Specify the state code
    "key": "9fe29c82d32372c3d1629631488612744bd7fe2f"  # Authentication key
}

# Send an API request to fetch PUMS data
response = requests.get(api_endpoint, params=params)

if response.status_code == 200:
    # If the request is successful, parse the JSON response and create a DataFrame
    data = response.json()
    data = pd.DataFrame(data[1:], columns=data[0])
else:
    print(f"API request failed with status code {response.status_code}")
    data = pd.DataFrame()

# Ensure all data columns are of integer type and drop rows with missing values
data = data.astype(int).dropna()

# Create a new categorical variable "LIF_CYC" based on "HHT" (Household Type)
data["LIF_CYC"] = data["HHT"].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ["F", "F", "T", "T", "T", "T", "T", "T", "F", "F", "F", "F"])

# Create a binary variable for "VEH" (Number of Vehicles)
data.loc[data["VEH"] >= 2, 'VEH'] = "multi"
data["VEH"] = data["VEH"].replace([0, 1, "multi"], ["F", "T", "T"])

# Create a categorical variable for "HINCP" (Household Income)
data["HINCP"] = pd.cut(data["HINCP"], [0, 75000, 150000, max(data["HINCP"])], labels=["L", "M", "H"])

# Create a binary variable for "SCHL" (Educational Attainment)
data["SCHL"] = pd.cut(data["SCHL"], [0, 1, 17, max(data["SCHL"])], labels=["no", "under", "high"])
data["SCHL"] = data["SCHL"].replace(["no", "under", "high"], ["F", "F", "T"])

# Create a binary variable for "AGEP" (Age)
data["AGEP"] = pd.cut(data["AGEP"], [0, 24, 44, 64, max(data["AGEP"])], labels=["young", "mid", "high", "senior"])
data["AGEP"] = data["AGEP"].replace(["young", "mid", "high", "senior"], ["F", "F", "F", "T"])

# Create a binary variable for "SEX" (Gender)
data["SEX"] = data["SEX"].replace([1, 2], ["F", "T"])

# Define categories for "c_cyc," "c_age," "c_edu," "c_sex," "c_car," and "c_inc"
c_cyc = ["T", "F"]
c_age = ["T", "F"]
c_edu = ["T", "F"]
c_sex = ["T", "F"]
c_car = ["T", "F"]
c_inc = ["L", "M", "H"]

# Define a function to generate a synthetic population based on data
def generate(data):
    mat = np.zeros((len(c_cyc), len(c_age), len(c_edu), len(c_sex), len(c_car), len(c_inc)))
    combined = [[a, b, c, d, f, g] for a in c_cyc for b in c_age for c in c_edu for d in c_sex for f in c_car for g in c_inc]
    for ind in combined:
        cyc, age, edu, sex, car, inc = ind
        try:
            mat[c_cyc.index(cyc), c_age.index(age), c_edu.index(edu), c_sex.index(sex), c_car.index(car), c_inc.index(inc)] = len(data[(data["LIF_CYC"] == cyc) & (data["AGEP"] == age) & (data["SCHL"] == edu) & (data["SEX"] == sex) & (data["VEH"] == car) & (data["HINCP"] == inc)])
        except:
            mat[c_cyc.index(cyc), c_age.index(age), c_edu.index(edu), c_sex.index(sex), c_car.index(car), c_inc.index(inc)] = 0
    return mat

# Generate the synthetic population matrix
mat = generate(data)

In [22]:
# Define a function to degenerate the IPF-generated matrix into a DataFrame
def degenerate(matt):
    ds = pd.DataFrame(columns=["age", "sex", "edu", "car", "cyc", "inc", "size"])
    combined = [[a, b, c, d, f, g] for a in c_age for b in c_sex for c in c_edu for d in c_car for f in c_cyc for g in c_inc]

    for i, ind in enumerate(combined):
        age, sex, edu, car, cyc, inc = ind
        ds.loc[i, "size"] = matt[c_cyc.index(cyc), c_age.index(age), c_edu.index(edu), c_sex.index(sex), c_car.index(car), c_inc.index(inc)]
        ds.loc[i, "cyc"] = cyc
        ds.loc[i, "age"] = age
        ds.loc[i, "edu"] = edu
        ds.loc[i, "sex"] = sex
        ds.loc[i, "car"] = car
        ds.loc[i, "inc"] = inc
    return ds

# Define a function to perform IPF (Iterative Proportional Fitting) for a given social factor
def ipfn_gen(name, matt):
    # Define aggregates and dimensions for IPF
    aggregates = [cyc_data[name].values, age_data[name].values, edu_data[name].values, sex_data[name].values, car_data[name].values, inc_data[name].values]
    dimensions = [[0], [1], [2], [3], [4], [5], [6]]
    # Create an IPF object and perform IPF iterations
    IPF = ipfn.ipfn(matt, aggregates, dimensions)
    m = IPF.iteration()
    return m * sum(sex_data[name].values) / np.sum(m)

# Generate population configurations for different social factors
result = degenerate(ipfn_gen(age_data.columns[-1], np.copy(mat)))
for col in tqdm(age_data.columns):
    try:
        result[col] = degenerate(ipfn_gen(col, np.copy(mat)))["size"].values
    except:
        continue

# Drop columns with NaN values and the 'size' column
result = result.dropna(axis='columns')
if 'size' in result.columns:
    result = result.drop('size', axis=1)

# Map categorical values to numerical values
original_x = result[["age", "sex", "edu", "car", "cyc", "inc"]].replace(["F", "T", "L", "M", "H"], [0, 1, 0, 1, 2])

# Save the population configurations to a CSV file
result.to_csv(base_dir + "Website/Result/" + area + "_population.csv", index=False)

/usr/local/lib/python3.10/dist-packages/ipfn/ipfn.py:146: RuntimeWarning: invalid value encountered in double_scalars
  if abs(m_ijk / ori_ijk - 1) > max_conv:


ipfn converged: convergence_rate not updating or below rate_tolerance


  0%|          | 0/501 [00:00<?, ?it/s]

ipfn converged: convergence_rate not updating or below rate_tolerance


  0%|          | 1/501 [00:00<01:10,  7.12it/s]

...

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


<ipython-input-22-6376f46dcee1>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = degenerate(ipfn_gen(col, np.copy(mat)))["size"].values
 99%|█████████▉| 498/501 [00:49<00:00,  8.78it/s]<ipython-input-22-6376f46dcee1>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = degenerate(ipfn_gen(col, np.copy(mat)))["size"].values
100%|█████████▉| 499/501 [00:49<00:00,  8.99it/s]<ipython-input-22-6376f46dcee1>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


/usr/local/lib/python3.10/dist-packages/ipfn/ipfn.py:146: RuntimeWarning: invalid value encountered in double_scalars
  if abs(m_ijk / ori_ijk - 1) > max_conv:
<ipython-input-22-6376f46dcee1>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[col] = degenerate(ipfn_gen(col, np.copy(mat)))["size"].values
100%|██████████| 501/501 [00:50<00:00, 10.00it/s]


ipfn converged: convergence_rate not updating or below rate_tolerance


## Delivery Estimation

In [23]:
# Read PSRC day and person data for different survey years
d = pd.read_csv(base_dir + "Data/PSRC_day.csv")
d17 = d[d["survey_year"] == 2017]
d19 = d[d["survey_year"] == 2019]
d21 = d[d["survey_year"] == 2021]

p = pd.read_csv(base_dir + "Data/PSRC_person.csv")
p17 = p[p["survey_year"] == 2017]
p19 = p[p["survey_year"] == 2019]
p21 = p[p["survey_year"] == 2021]

# Define a function to process the combined person and day data
def process_data(p, d):
    data = p.merge(d, on="person_id", how="outer")

    # Remove rows with missing or "Skip Logic" values for delivery frequency
    data = data.dropna(subset=["delivery_pkgs_freq", "delivery_grocery_freq", "delivery_food_freq", "delivery_work_freq"])
    data = data[data["delivery_pkgs_freq"] != "Missing: Skip Logic"]
    data = data[data["delivery_grocery_freq"] != "Missing: Skip Logic"]
    data = data[data["delivery_food_freq"] != "Missing: Skip Logic"]
    data = data[data["delivery_work_freq"] != "Missing: Skip Logic"]

    # Map categorical delivery frequency values to numeric values
    data['delivery_pkgs_freq'] = data['delivery_pkgs_freq'].replace(['0 (none)', '1', '2', '3', '4', '5 or more'], [0, 1, 2, 3, 4, 5])
    data['delivery_grocery_freq'] = data['delivery_grocery_freq'].replace(['0 (none)', '1', '2', '3', '4', '5 or more'], [0, 1, 2, 3, 4, 5])
    data['delivery_food_freq'] = data['delivery_food_freq'].replace(['0 (none)', '1', '2', '3', '4', '5 or more'], [0, 1, 2, 3, 4, 5])
    data['delivery_work_freq'] = data['delivery_work_freq'].replace(['0 (none)', '1', '2', '3', '4', '5 or more'], [0, 1, 2, 3, 4, 5])

    # Create a new column "delivery" by summing the delivery frequency values
    data['delivery'] = data['delivery_pkgs_freq'] + data['delivery_grocery_freq'] + data['delivery_food_freq']

    # Map categorical values to binary values for other features
    data.loc[data['vehicle_count'] == '0 (no vehicles)', 'vehicle_count'] = 0
    data.loc[data['vehicle_count'] != 0, 'vehicle_count'] = 1

    data.loc[data['gender'] == 'Female', 'gender'] = 1
    data.loc[data['gender'] != 1, 'gender'] = 0

    data.loc[(data['lifecycle'] == 'Household includes children age 5-17') | (data['lifecycle'] == 'Household includes children under 5'), 'lifecycle'] = 1
    data.loc[data['lifecycle'] != 1, 'lifecycle'] = 0

    data.loc[(data['hhincome_detailed'] == '$150,000-$199,999') | (data['hhincome_detailed'] == '$200,000-$249,999') | (data['hhincome_detailed'] == '$250,000 or more'), 'hhincome_detailed'] = 2
    data.loc[(data['hhincome_detailed'] == '$75,000-$99,999') | (data['hhincome_detailed'] == '$100,000-$149,999'), 'hhincome_detailed'] = 1
    data.loc[(data['hhincome_detailed'] != 1) & (data['hhincome_detailed'] != 2), 'hhincome_detailed'] = 0

    data.loc[(data['age'] == '65-74 years') | (data['age'] == '75-84 years') | (data['age'] == '85 or years older'), 'age'] = 1
    data.loc[data['age'] != 1, 'age'] = 0

    data.loc[(data['education'] == 'Bachelor degree') | (data['education'] == 'Graduate/post-graduate degree') | (data['education'] == 'Some college'), 'education'] = 1
    data.loc[data['education'] != 1, 'education'] = 0

    # Select and rename relevant columns for analysis
    data = data[['age', 'gender', 'education', 'vehicle_count', 'lifecycle', 'hhincome_detailed', 'delivery']]
    data.columns = ["age", "sex", "edu", "car", "cyc", "inc", 'delivery']

    return data

# Process the data for different survey years (2017, 2019, 2021)
data17 = process_data(p17, d17)
data19 = process_data(p19, d19)
data21 = process_data(p21, d21)

# Separate input features and target variable for 2017 data
x = data17.drop(columns="delivery")
y = data17[["delivery"]]

# Train a Gradient Boosting Regressor model for 2017 data
reg17 = GradientBoostingRegressor(random_state=0)
reg17.fit(x, y)

# Separate input features and target variable for 2019 data
x = data19.drop(columns="delivery")
y = data19[["delivery"]]

# Train a Gradient Boosting Regressor model for 2019 data
reg19 = GradientBoostingRegressor(random_state=0)
reg19.fit(x, y)

# Separate input features and target variable for 2021 data
x = data21.drop(columns="delivery")
y = data21[["delivery"]]

# Train a Gradient Boosting Regressor model for 2021 data
reg21 = GradientBoostingRegressor(random_state=0)
reg21.fit(x, y)

<ipython-input-23-7bb246035564>:2: DtypeWarning: Columns (16,17,23,25,34,36,37,38,39,40,41,42,43,46,47,66,67,68,69,70,71,72,73,74,75,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(base_dir + "Data/PSRC_day.csv")
<ipython-input-23-7bb246035564>:7: DtypeWarning: Columns (1,3,7,23,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,77,78,81,84,85,95,96,97,98,99,100,101,102,103,104,105,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,135,136,137,138,139,140,141,142,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  p = pd.read_csv(base_dir + "Data/PSRC_person.csv")
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py

GradientBoostingRegressor(random_state=0)

In [25]:
# Read NHTS household and person data for the specified CBSA region and other regions
hh = pd.read_csv(base_dir + "Data/NHTS_household.csv")
per = pd.read_csv(base_dir + "Data/NHTS_person.csv")
hh_region = hh[hh["HH_CBSA"] == cbsa_code]
hh_other = hh[hh["HH_CBSA"] != cbsa_code]

# Define a function to extract delivery data from person and household data
def get_delivery_set(per_data, hh_data):
    d = per_data.merge(hh_data, on="HOUSEID")[["LIF_CYC_y", "R_AGE", "EDUC", "R_SEX", "HHVEHCNT_y", "R_RACE", "HHFAMINC_y", "HHSIZE_y", "DELIVER"]]
    d = d.rename(columns={"LIF_CYC_y": "LIF_CYC", "HHVEHCNT_y": "HHVEHCNT", "HHFAMINC_y": "HHFAMINC", "HHSIZE_y": "HHSIZE"})
    data_del = d.drop(columns=["HHSIZE", "R_RACE"]).dropna()

    # Map categorical values to binary values for "cyc" and "car"
    data_del["cyc"] = data_del["LIF_CYC"].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ["T", "F", "T", "F", "F", "T", "F", "F", "F", "T"])
    data_del.loc[data_del["HHVEHCNT"] >= 2, 'HHVEHCNT'] = "multi"
    data_del["car"] = data_del["HHVEHCNT"].replace([0, 1, "multi"], ["F", "T", "T"])

    # Create "inc," "edu," "age," and "sex" columns with specific bin values
    data_del["inc"] = pd.cut(data_del["HHFAMINC"], [0, 6, 9, 11], labels=["L", "M", "H"])
    data_del["edu"] = pd.cut(data_del["EDUC"], [-10, 2, 5], labels=["F", "T"])
    data_del["age"] = pd.cut(data_del["R_AGE"], [0, 64, max(data_del["R_AGE"])], labels=["F", "T"])
    data_del["sex"] = pd.cut(data_del["R_SEX"], [0, 1, 2], labels=["F", "T"])
    data_del = data_del.dropna()

    # Remove rows with negative delivery values
    data_del = data_del[data_del["DELIVER"] >= 0]

    X = data_del[["age", "sex", "edu", "car", "cyc", "inc"]].replace(["F", "T", "L", "M", "H"], [0, 1, 0, 1, 2]).astype(int)
    Y = data_del[["DELIVER"]]
    return X, Y

# Extract delivery data for the CBSA region and other regions
xN, yN = get_delivery_set(per, hh_other)
xn, yn = get_delivery_set(per, hh_region)

# Define a function to perform Bayesian data aggregation
def bayesian_data(xN, yN, xn, yn):
    combined_df = pd.concat([xn, yn], axis=1)
    grouped = combined_df.groupby(list(xn.columns))
    result = grouped.agg({'DELIVER': ['count', 'mean', 'var']}).reset_index()
    result = result[result[('DELIVER', 'count')] > 1]
    result.columns = [''.join(col).strip() for col in result.columns.values]
    unique_result_combinations = set(map(tuple, result[['age', 'sex', 'edu', 'car', 'cyc', 'inc']].values))
    filter_condition = xN.apply(tuple, axis=1).isin(unique_result_combinations)
    xN_filtered = xN[filter_condition]
    yN_filtered = yN[filter_condition]
    dataN = pd.concat([xN_filtered, yN_filtered], axis=1)
    final_result = dataN.merge(result, on=['age', 'sex', 'edu', 'car', 'cyc', 'inc'])
    mu0 = final_result['DELIVER'].values
    var0 = np.var(final_result[['DELIVER']].values)
    mu = final_result['DELIVERmean'].values
    var = final_result['DELIVERvar'].values
    n = final_result['DELIVERcount'].values
    final_result["DELIVER"] = (1 / (1 / var0 ** 2 + n / var ** 2) * (mu0 / var0 ** 2 + n * mu / var ** 2)).reshape(-1, 1)
    final_result = final_result.dropna()
    x = pd.concat([final_result[['age', 'sex', 'edu', 'car', 'cyc', 'inc']], xn], axis=0).reset_index().drop(columns="index")
    y = pd.concat([final_result[['DELIVER']], yn], axis=0).reset_index().drop(columns="index")
    return x, y

# Perform Bayesian data aggregation
x, y = bayesian_data(xN, yN, xn, yn)

# Train an XGBoost Regressor model using the aggregated data
model = xgb.XGBRegressor()
model.fit(x, y)

# Synthesize delivery values for the census tracts and combine them with previous data
result = model.predict(original_x) + reg19.predict(original_x) - reg17.predict(original_x)
result[result < 0] = 0
deliver_result = original_x.copy()
deliver_result['result'] = result
population_result = pd.read_csv(base_dir + "Website/Result/" + area + "_population.csv").drop(["age", "sex", "edu", "car", "cyc", "inc"], axis=1)
deliver_result['total'] = np.sum(population_result.values, axis=1)

# Save the results to a CSV file
deliver_result.to_csv(base_dir + "Website/Result/" + area + '_prediction.csv', index=False)

<ipython-input-25-7c13326379ba>:54: RuntimeWarning: divide by zero encountered in divide
  final_result["DELIVER"] = (1 / (1 / var0 ** 2 + n / var ** 2) * (mu0 / var0 ** 2 + n * mu / var ** 2)).reshape(-1, 1)
<ipython-input-25-7c13326379ba>:54: RuntimeWarning: invalid value encountered in divide
  final_result["DELIVER"] = (1 / (1 / var0 ** 2 + n / var ** 2) * (mu0 / var0 ** 2 + n * mu / var ** 2)).reshape(-1, 1)


## Combine for Result

In [26]:
# Ignore warning messages
warnings.filterwarnings('ignore')

# Load the synthetic population and prediction data
population = pd.read_csv(base_dir + "Website/Result/" + area + "_population.csv").drop(["age", "sex", "edu", "car", "cyc", "inc"], axis=1)
prediction = pd.read_csv(base_dir + "Website/Result/" + area + "_prediction.csv")

# Calculate the variance of the prediction results
var = np.var(prediction["result"])

# Initialize an empty DataFrame to store confidence interval results
CTresult = pd.DataFrame()

# Iterate through columns in the population data
for col in tqdm(population.columns):
    # Calculate the confidence interval
    diff = var * 1.96 / np.sqrt(np.sum(population[col].values))
    lower = np.sum(np.floor(population[col].values) * np.floor(prediction["result"].values - diff)).astype(int)
    upper = np.sum(np.ceil(population[col].values) * np.ceil(prediction["result"].values + diff)).astype(int)
    population_whole = np.sum(population[col].values).astype(int)
    col = col.replace(", ", "-")

    # Store the lower, upper, and total population values in the DataFrame
    CTresult[col] = np.array([lower, upper, population_whole])

# Remove columns with missing values
CTresult = CTresult.dropna(axis=1)

# Save the confidence interval results to a CSV file
CTresult.to_csv(base_dir + "Website/Result/" + area + "_CTresult.csv", index=False)

100%|██████████| 498/498 [00:00<00:00, 549.97it/s]


# 2021

## Synthetic Population

In [27]:
# Define the API URL and parameters. Use those to get marginal sums for different social factors.

# Extract data related to population characteristics (age) using the Census API.
df = extract_census_data("https://api.census.gov/data/2021/acs/acs5/subject", "S0101", region_df)
data = df.set_index("NAME")

# Calculate the total population and the population that identifies as female.
data["T"] = (data[["S0101_C01_015E"]].values).astype(int) + (data[["S0101_C01_016E"]].values).astype(int) + \
            (data[["S0101_C01_017E"]].values).astype(int) + (data[["S0101_C01_018E"]].values).astype(int) + \
            (data[["S0101_C01_019E"]].values).astype(int)
data["F"] = (data[["S0101_C01_001E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Create a DataFrame for age-related data.
age_data = data[["T", "F"]].T

# Extract data related to population characteristics (sex) using the Census API.
df = extract_census_data("https://api.census.gov/data/2021/acs/acs5/subject", "S1101", region_df)
data = df.set_index("NAME")

# Calculate the total population and the population that identifies as female.
data["T"] = (data[["S1101_C01_005E"]].values).astype(int)
data["F"] = (data[["S1101_C01_001E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Create a DataFrame for sex-related data.
sex_data = data[["T", "F"]].T

# Extract data related to population characteristics (household type) using the Census API.
df = extract_census_data("https://api.census.gov/data/2021/acs/acs5/subject", "S1501", region_df)
data = df.set_index("NAME")

# Calculate the total population and the population in households with children.
data["T"] = (data[["S1501_C01_005E"]].values).astype(int) + (data[["S1501_C01_015E"]].values).astype(int)
data["F"] = (data[["S1501_C01_001E"]].values).astype(int) + (data[["S1501_C01_006E"]].values).astype(int) - (data[["T"]].values).astype(int)

# Create a DataFrame for household type (children or not) related data.
edu_data = data[["T", "F"]].T

# Extract data related to income and education using the Census API.
df = extract_census_data("https://api.census.gov/data/2021/acs/acs5/subject", "S1901", region_df)
data = df.set_index("NAME")

# Calculate high, medium, and low-income populations based on education levels.
data["H"] = (data[["S1901_C01_001E"]].values).astype(int) * ((data[["S1901_C01_010E"]].values).astype(float) + \
                                                           (data[["S1901_C01_011E"]].values).astype(float)) / 100
data["M"] = (data[["S1901_C01_001E"]].values).astype(int) * ((data[["S1901_C01_008E"]].values).astype(float) + \
                                                           (data[["S1901_C01_009E"]].values).astype(float)) / 100
data["L"] = (data[["S1901_C01_001E"]].values).astype(int) - (data[["H"]].values).astype(float)

# Create a DataFrame for income-related data.
inc_data = data[["L", "M", "H"]].T

# Extract data related to transportation (car ownership) using the Census API.
df = extract_census_data("https://api.census.gov/data/2021/acs/acs5", "B08014", region_df)
data = df.set_index("NAME")

# Calculate the total population with and without cars.
data["T"] = (data[["B08014_001E"]].values).astype(int) - (data[["B08014_002E"]].values).astype(int)
data["F"] = (data[["B08014_002E"]].values).astype(int)

# Create a DataFrame for car ownership-related data.
car_data = data[["T", "F"]].T

100%|██████████| 15/15 [00:11<00:00,  1.27it/s]


In [28]:
# Step 1: Make an API request to the U.S. Census Bureau's ACS dataset for 2019.
api_endpoint = "https://api.census.gov/data/2021/acs/acs1/pums"
params = {
    "get": "SEX,SCHL,HHT,VEH,HINCP,AGEP",
    "for": "state:"+state_codes_str,
    "key": "9fe29c82d32372c3d1629631488612744bd7fe2f"
}
response = requests.get(api_endpoint, params=params)

# Check if the API request was successful
if response.status_code == 200:
    # Convert the response data into a Pandas DataFrame
    data = response.json()
    data = pd.DataFrame(data[1:], columns=data[0])
else:
    print(f"API request failed with status code {response.status_code}")
    data = pd.DataFrame()

# Convert data to integer values and drop rows with missing values
data = data.astype(int).dropna()

# Create a new column "LIF_CYC" based on the "HHT" (household type) column
data["LIF_CYC"] = data["HHT"].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ["F", "F", "T", "T", "T", "T", "T", "T", "F", "F", "F", "F"])

# Categorize vehicle ownership into "multi" for 2 or more vehicles, "T" for 1 vehicle, and "F" for no vehicle
data.loc[data["VEH"] >= 2, 'VEH'] = "multi"
data["VEH"] = data["VEH"].replace([0, 1, "multi"], ["F", "T", "T"])

# Categorize income into "L" (low), "M" (medium), and "H" (high) based on predefined income thresholds
data["HINCP"] = pd.cut(data["HINCP"], [0, 75000, 150000, max(data["HINCP"])], labels=["L", "M", "H"])

# Categorize education ("SCHL") into "no" (no schooling), "under" (undergraduate education), and "high" (high school or above)
data["SCHL"] = pd.cut(data["SCHL"], [0, 1, 17, max(data["SCHL"])], labels=["no", "under", "high"])
data["SCHL"] = data["SCHL"].replace(["no", "under", "high"], ["F", "F", "T"])

# Categorize age ("AGEP") into "young," "mid," "high," and "senior"
data["AGEP"] = pd.cut(data["AGEP"], [0, 24, 44, 64, max(data["AGEP"])], labels=["young", "mid", "high", "senior"])
data["AGEP"] = data["AGEP"].replace(["young", "mid", "high", "senior"], ["F", "F", "F", "T"])

# Categorize sex ("SEX") into "F" (Female) and "T" (Male)
data["SEX"] = data["SEX"].replace([1, 2], ["F", "T"])

# Define categories for the attributes
c_cyc = ["T", "F"]
c_age = ["T", "F"]
c_edu = ["T", "F"]
c_sex = ["T", "F"]
c_car = ["T", "F"]
c_inc = ["L", "M", "H"]

# Generate a synthetic population matrix based on the defined categories and ACS data
mat = generate(data)

# Step 3: Generate a synthetic population for the latest year (2021) based on the generated matrix
result = degenerate(ipfn_gen(age_data.columns[-1], np.copy(mat)))

# Iterate over each social factor (age, education, etc.) to create synthetic populations
for col in tqdm(age_data.columns):
    try:
        result[col] = degenerate(ipfn_gen(col, np.copy(mat)))["size"].values
    except:
        continue

# Remove columns with NaN values and the "size" column (if present)
result = result.dropna(axis='columns')
if 'size' in result.columns:
    result = result.drop('size', axis=1)

# Create the original_x DataFrame based on the attributes (age, sex, edu, car, cyc, inc)
original_x = result[["age", "sex", "edu", "car", "cyc", "inc"]].replace(["F", "T", "L", "M", "H"], [0, 1, 0, 1, 2])

# Save the generated population data for 2021 as a CSV file
result.to_csv(base_dir + "Website/Result/" + area + "_population_2021.csv", index=False)

ipfn converged: convergence_rate not updating or below rate_tolerance


  0%|          | 0/550 [00:00<?, ?it/s]

ipfn converged: convergence_rate not updating or below rate_tolerance


  0%|          | 1/550 [00:00<01:05,  8.41it/s]

ipfn converged: convergence_rate not updating or below rate_tolerance


...

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


 99%|█████████▊| 542/550 [00:38<00:00, 26.28it/s]

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


100%|██████████| 550/550 [00:38<00:00, 14.20it/s]

ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance
ipfn converged: convergence_rate not updating or below rate_tolerance


## Delivery Estimation - Generate Data First

In [29]:
# Load the population and prediction data for 2021 and replace categorical labels with numeric values
pop = pd.read_csv(base_dir + "Website/Result/" + area + "_population.csv").replace(["T", "F", "L", "M", "H"], [1, 0, 0, 1, 2])
pred = pd.read_csv(base_dir + "Website/Result/" + area + "_prediction.csv")

# Merge population and prediction data on the attributes (age, sex, edu, car, cyc, inc)
data19 = pop.merge(pred, on=["age", "sex", "edu", "car", "cyc", "inc"])[["age", "sex", "edu", "car", "cyc", "inc", "result"]]

# Calculate the total population for 2019
data19["total_pop"] = np.sum(pd.read_csv(base_dir + "Website/Result/" + area + "_population.csv").drop(columns=["age", "sex", "edu", "car", "cyc", "inc"]), axis=1)

# Define the initial state parameters
start = np.mean(data19["result"].values)
stddev = np.std(data19["result"].values)

# Metropolis-Hastings sampling function
def metropolis_hastings(n, avg, start, proposal_stddev):
    samples = [start]
    current_value = start
    for _ in range(n):
        proposed_value = current_value + np.random.normal(0, proposal_stddev)
        likelihood_current = -((current_value - avg) ** 2) / 2
        likelihood_proposed = -((proposed_value - avg) ** 2) / 2
        acceptance_ratio = np.exp(likelihood_proposed - likelihood_current)
        if np.random.rand() < acceptance_ratio:
            current_value = proposed_value
        samples.append(current_value)
    return samples

# Create a DataFrame to store the synthetic data for 2019
df_19 = pd.DataFrame()

# Iterate over each row in data19 and apply Metropolis-Hastings to generate synthetic data
for row in tqdm(data19.iterrows(), miniters=0):
    n = row[1]["total_pop"].astype(int)
    avg = row[1]["result"]
    samples = metropolis_hastings(n, avg, start, stddev)
    df = pd.DataFrame(samples, columns=["DELIVER"])
    df[["age", "sex", "edu", "car", "cyc", "inc"]] = row[1][["age", "sex", "edu", "car", "cyc", "inc"]]
    df_19 = pd.concat([df_19, df], axis=0)

# Separate the features (x) and the synthetic target (y)
x = df_19.drop(columns=['DELIVER'])
y = df_19[["DELIVER"]]

96it [00:12,  7.92it/s]


In [30]:
# Create an XGBoost regressor model
model = xgb.XGBRegressor()
model.fit(x, y)

# Synthesize data for 2021 based on the previous data
result = model.predict(original_x) + reg21.predict(original_x) - reg19.predict(original_x)

# Ensure that the synthesized delivery values are non-negative
result[result < 0] = 0

# Create a DataFrame to store the synthesized data
deliver_result = original_x.copy()
deliver_result['result'] = result

# Load the population result for 2021 and calculate the total population
population_result = pd.read_csv(base_dir + "Website/Result/" + area + "_population_2021.csv").drop(["age", "sex", "edu", "car", "cyc", "inc"], axis=1)
deliver_result['total'] = np.sum(population_result.values, axis=1)

# Save the synthesized data to a CSV file
deliver_result.to_csv(base_dir + "Website/Result/" + area + '_prediction_2021.csv', index=False)

## Combine for Results

In [31]:
# Suppress warnings
warnings.filterwarnings('ignore')

# Load the population and prediction data for 2021
population = pd.read_csv(base_dir + "Website/Result/" + area + "_population_2021.csv").drop(["age", "sex", "edu", "car", "cyc", "inc"], axis=1)
prediction = pd.read_csv(base_dir + "Website/Result/" + area + "_prediction_2021.csv")

# Calculate the variance of the prediction results
var = np.var(prediction["result"])

# Create a DataFrame to store the prediction intervals
CTresult = pd.DataFrame()

# Iterate over columns (census tracts) to calculate prediction intervals
for col in tqdm(population.columns):
    # Calculate the standard error for the prediction
    diff = var * 1.96 / np.sqrt(np.sum(population[col].values))

    # Calculate lower and upper bounds for the prediction interval
    lower = np.sum(np.floor(population[col].values) * np.floor(prediction["result"].values - diff)).astype(int)
    upper = np.sum(np.ceil(population[col].values) * np.ceil(prediction["result"].values + diff)).astype(int)

    # Calculate the total population for the column
    population_whole = np.sum(population[col].values).astype(int)

    # Replace any commas and spaces in the column name
    col = col.replace(", ", "-")

    # Store the lower, upper, and total population in the CTresult DataFrame
    CTresult[col] = np.array([lower, upper, population_whole])

# Remove any columns with missing values
CTresult = CTresult.dropna(axis=1)

# Save the prediction intervals to a CSV file
CTresult.to_csv(base_dir + "Website/Result/" + area + "_CTresult_2021.csv", index=False)

100%|██████████| 438/438 [00:00<00:00, 1743.97it/s]
